In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
import pandas
from tqdm import tqdm

In [ ]:
tf.config.list_physical_devices()

In [ ]:
data = pandas.read_csv("bazaar-challenge-data/installed_apps.csv")
data_related = pandas.read_csv("bazaar-challenge-data/related_apps.csv")

Hyperparameters

In [ ]:
steps_per_epoch = 1000
steps_per_val_epoch = 200
runs_dir = "Runs/"
model_name = "batch_size_+1_train"
model_dir = runs_dir + model_name + "/"
Epochs = 150
num_classes = 45231
last_epoch = -1

In [ ]:
length = {} # number of rows for each length
rows_of_length_n = [[] for i in range(100)] # rows with a length of n
length_list = [] # length of each row

for i in range(100):
    length[i] = 0
for i in tqdm(range(1000000)):
    length_list.append(len([int(i.replace('\'', '')) for i in data['installed_apps'][i][1:-1].split(',')]))
    length[len([int(i.replace('\'', '')) for i in data['installed_apps'][i][1:-1].split(',')])] += 1

for i in range(1000000):
    rows_of_length_n[length_list[i]].append(i)

In [ ]:
lens = np.zeros(100)
for i in length:
    lens[i] = length[i]
plt.plot(lens)

Defining the model

In [ ]:
x_inp = tf.keras.layers.Input(shape=(None,1))
L1 = (keras.layers.LSTM(256, activation='relu', return_sequences=True))(x_inp)
L2 = (keras.layers.LSTM(256, activation='relu', return_sequences=True))(L1)
L3 = (keras.layers.LSTM(128, activation='relu', return_sequences=False))(L2)
D1 = (keras.layers.Dense(100, activation='relu'))(L3)
D2 = (keras.layers.Dense(80, activation='relu'))(D1)
Out = (keras.layers.Dense(45231, activation='softmax'))(D2)
model = tf.keras.models.Model(inputs=[x_inp], outputs=[Out])
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

Data generator(returns rows with length size-1 as x and returns the last element as y)

In [ ]:
def numpy_by_size(size):
    x = np.zeros((length[size],size-1,1), dtype = np.uint16)
    y = np.zeros((length[size],1,1), dtype = np.uint16)
    
    rows = rows_of_length_n[size]
    for i in range(length[size]):
        
        this_row = np.array([int(row.replace('\'', '')) for row in data['installed_apps'][rows[i]][1:-1].split(',')])        
        x[i] = np.expand_dims(this_row[:-1],1)
        y[i] = np.expand_dims(this_row[-1],0)
        
    return x, y

In [ ]:
def test_numpy_by_size(size):
    x = np.zeros((length[size],size,1), dtype = np.uint16)
    
    
    rows = rows_of_length_n[size]
    for i in range(length[size]):
        
        this_row = np.array([int(row.replace('\'', '')) for row in data['installed_apps'][rows[i]][1:-1].split(',')])        
        x[i] = np.expand_dims(this_row,1)
        
    return x

Training and testing Loop

In [ ]:
l = [[] for i in range(1000000)]

for i in tqdm(range(1000000)):
    l[i].append(i)

for i in tqdm(range(2,101)):
    if i < 35:
        features, target = numpy_by_size(i)
        print("Training with size", i ,":")
        model.fit(  features,
                    target,
                    validation_data=valgenerator_1,
                    batch_size = 20,
                    epochs=1,
                    shuffle = True,
                    # steps_per_epoch=length[i],
                    validation_steps=steps_per_val_epoch)
                    # validation_split = 0.1)
        model.save(model_dir + model_name + "_size_" + str(i) + ".h5")
    
    test = test_numpy_by_size(i - 1)
    pred = model.predict(test, verbose=False)

    rows = rows_of_length_n[i - 1]
    for k in range(length[i-1]):
        pred_row = pred[k]
        j = 0
        lis = [int(row.replace('\'', '')) for row in data['installed_apps'][rows[k]][1:-1].split(',')]
        while j < 10:
            if not(np.argmax(pred_row) in lis):
                l[rows[k]].append(np.argmax(pred_row))
                j += 1
            pred_row[np.argmax(pred_row)] = 0

In [ ]:
relation = [[] for i in range(45231)]
for i in range(177745):
    relation[data_related['src_app_id'][i]].append([data_related['dst_app_id'][i],data_related['installs'][i]])

In [ ]:
for i in tqdm(range(1000000)):
    row_list = [int(row.replace('\'', '')) for row in data['installed_apps'][i][1:-1].split(',')]
    if len(row_list) == 1:
        last = row_list[-1]
        tes = len(l[last])
        which = 0
        for j in range(min(tes,4)):
            if l[last][0][1] > 50 and not(l[last][0][0] in relation[i]):
                relation[i][9-which] = l[last][j][0]
                which += 1
    else:
        last = row_list[-1]
        second = row_list[-2]
        tes = len(l[last])
        tes2 = len(l[second])
        which = 0
        for j in range(min(tes,4)):
            if l[last][0][1] > 50 and not(l[last][j][0] in relation[i]):
                relation[i][which] = l[last][j][0]
                which += 1
        for j in range(min(tes2,2)):
            if l[second][0][1] > 100 and not(l[second][j][0] in relation[i]):
                relation[i][which] = l[second][j][0]
                which += 1
    


In [ ]:
header = ['user_id','app_id0','app_id1','app_id2','app_id3','app_id4','app_id5','app_id6','app_id7','app_id8','app_id9']
send_file_data = pandas.DataFrame(l, columns=header)
send_file_data.to_csv('result.csv', index=False)